In [59]:
import pandas as pd
import os
import torch
import numpy as np
import sys; sys.path.append('..')
import matplotlib
import requests
import utils.constant as constant

from tqdm.notebook import tqdm, tqdm_notebook
from datetime import datetime, timedelta

from IPython.display import clear_output

path_dataset = constant.PATH_MIMIC_III
path_ddi_dataset = constant.PATH_DDI_DATA

list_csvgz = [f for f in os.listdir(path_dataset) if f[-6:]=="csv.gz"]
list_csvgz

['DRGCODES.csv.gz',
 'LABEVENTS.csv.gz',
 'PATIENTS.csv.gz',
 'D_ITEMS.csv.gz',
 'DIAGNOSES_ICD.csv.gz',
 'CAREGIVERS.csv.gz',
 'SERVICES.csv.gz',
 'CPTEVENTS.csv.gz',
 'PRESCRIPTIONS_NCV_SOLVED.csv.gz',
 'PRESCRIPTIONS_PREPROCESSED.csv.gz',
 'OUTPUTEVENTS.csv.gz',
 'D_LABITEMS.csv.gz',
 'D_ICD_DIAGNOSES.csv.gz',
 'LABEVENTS_NEW_remove_duplicate_edges.csv.gz',
 'CALLOUT.csv.gz',
 'PRESCRIPTIONS_TIMESTEP_SOLVED.csv.gz',
 'ADMISSIONS.csv.gz',
 'D_CPT.csv.gz',
 'INPUTEVENTS_MV.csv.gz',
 'D_LABITEMS_NEW.csv.gz',
 'PROCEDUREEVENTS_MV.csv.gz',
 'ADMISSIONS_NEW.csv.gz',
 'PRESCRIPTIONS.csv.gz',
 'NOTEEVENTS.csv.gz',
 'DATETIMEEVENTS.csv.gz',
 'MICROBIOLOGYEVENTS.csv.gz',
 'ICUSTAYS.csv.gz',
 'DRUGS_NDC_FEAT.csv.gz',
 'D_ICD_PROCEDURES.csv.gz',
 'PRESCRIPTIONS_DURATION_SOLVED.csv.gz',
 'LABEVENTS_NEW.csv.gz',
 'CHARTEVENTS.csv.gz',
 'TRANSFERS.csv.gz',
 'PROCEDURES_ICD.csv.gz',
 'LABEVENTS_PREPROCESSED.csv.gz',
 'INPUTEVENTS_CV.csv.gz']

In [4]:
df_admissions = pd.read_csv(os.path.join(path_dataset, "ADMISSIONS.csv.gz"))

# Convert columns to datetime
df_admissions["ADMITTIME"] = pd.to_datetime(df_admissions["ADMITTIME"], format="%Y-%m-%d %H:%M:%S")
df_admissions["DISCHTIME"] = pd.to_datetime(df_admissions["DISCHTIME"], format="%Y-%m-%d %H:%M:%S")

df_admissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58976 entries, 0 to 58975
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ROW_ID                58976 non-null  int64         
 1   SUBJECT_ID            58976 non-null  int64         
 2   HADM_ID               58976 non-null  int64         
 3   ADMITTIME             58976 non-null  datetime64[ns]
 4   DISCHTIME             58976 non-null  datetime64[ns]
 5   DEATHTIME             5854 non-null   object        
 6   ADMISSION_TYPE        58976 non-null  object        
 7   ADMISSION_LOCATION    58976 non-null  object        
 8   DISCHARGE_LOCATION    58976 non-null  object        
 9   INSURANCE             58976 non-null  object        
 10  LANGUAGE              33644 non-null  object        
 11  RELIGION              58518 non-null  object        
 12  MARITAL_STATUS        48848 non-null  object        
 13  ETHNICITY       

# PRESCRIPTIONS

In [5]:
df_prescriptions = pd.read_csv(os.path.join(path_dataset, "PRESCRIPTIONS.csv.gz"))
df_prescriptions["STARTDATE"] = pd.to_datetime(df_prescriptions["STARTDATE"], format="%Y-%m-%d %H:%M:%S")
df_prescriptions["ENDDATE"]   = pd.to_datetime(df_prescriptions["ENDDATE"],   format="%Y-%m-%d %H:%M:%S")

# Among all recorded drug codes, the NDC code has the smallest number of `nan` values.
df_prescriptions = df_prescriptions[df_prescriptions.NDC.notnull()]
df_prescriptions = df_prescriptions.loc[df_prescriptions.STARTDATE.notnull() &
                                        df_prescriptions.ENDDATE.notnull()]
df_prescriptions = df_prescriptions.loc[df_prescriptions.STARTDATE < df_prescriptions.ENDDATE]

# Avoid the same drug being considered different due to different capitalization.
#     Many tutorials you’ll find only will tell you to pass in 'str' as the argument. 
#     While this holds true for versions of Pandas lower than 1.0, if you’re using 1.0 or later, pass in 'string' instead.
#     v.i.a. <https://datagy.io/pandas-column-values-to-strings/>
df_prescriptions['DRUG']           = df_prescriptions['DRUG'          ].astype("string").str.lower()
df_prescriptions["DRUG_TYPE"]      = df_prescriptions["DRUG_TYPE"     ].astype("string").str.upper()
df_prescriptions["PROD_STRENGTH"]  = df_prescriptions["PROD_STRENGTH" ].astype("string").str.lower()
df_prescriptions["DOSE_UNIT_RX"]   = df_prescriptions["DOSE_UNIT_RX"  ].astype("string").str.lower()
df_prescriptions["FORM_UNIT_DISP"] = df_prescriptions["FORM_UNIT_DISP"].astype("string").str.lower()
df_prescriptions["ROUTE"]          = df_prescriptions["ROUTE"         ].astype("string").str.upper()

df_prescriptions.info()

/data/data2/041/envs/LERS2/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3269: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4151987 entries, 0 to 4156449
Data columns (total 19 columns):
 #   Column             Dtype         
---  ------             -----         
 0   ROW_ID             int64         
 1   SUBJECT_ID         int64         
 2   HADM_ID            int64         
 3   ICUSTAY_ID         float64       
 4   STARTDATE          datetime64[ns]
 5   ENDDATE            datetime64[ns]
 6   DRUG_TYPE          string        
 7   DRUG               object        
 8   DRUG_NAME_POE      object        
 9   DRUG_NAME_GENERIC  object        
 10  FORMULARY_DRUG_CD  object        
 11  GSN                object        
 12  NDC                float64       
 13  PROD_STRENGTH      string        
 14  DOSE_VAL_RX        object        
 15  DOSE_UNIT_RX       string        
 16  FORM_VAL_DISP      object        
 17  FORM_UNIT_DISP     string        
 18  ROUTE              string        
dtypes: datetime64[ns](2), float64(2), int64(3), object(7), string(5

In [6]:
df_prescriptions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11,2175-06-12,MAIN,tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg capsule,2,mg,2,cap,PO
1,2214775,6,107064,NaN,2175-06-11,2175-06-12,MAIN,warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg tablet,5,mg,1,tab,PO
2,2215524,6,107064,NaN,2175-06-11,2175-06-12,MAIN,heparin sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit premix bag","25,000",unit,1,bag,IV
3,2216265,6,107064,NaN,2175-06-11,2175-06-12,BASE,d5w,NaN,NaN,HEPBASE,NaN,0.0,heparin base,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11,2175-06-12,MAIN,furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg tablet,20,mg,1,tab,PO


## Wrong start and end timestamps

This will cause `nan` values to appear when adding time steps later.

In [7]:
df_prescriptions[df_prescriptions.ROW_ID == 1497964]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
701226,1497964,9895,100006,NaN,2108-04-21,2108-04-18,MAIN,prednisone,Prednisone,Prednisone,PRED10,006749,54872525.0,10mg tab,30,mg,3,tab,PO


In [8]:
df_prescriptions = df_prescriptions.loc[df_prescriptions.STARTDATE.notnull() & 
                                        df_prescriptions.ENDDATE.notnull()]
df_prescriptions = df_prescriptions.loc[df_prescriptions.STARTDATE < df_prescriptions.ENDDATE]
df_prescriptions[df_prescriptions.HADM_ID == 100006]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
701194,1497955,9895,100006,291788.0,2108-04-12,2108-04-18,MAIN,risperidone,Risperidone,Risperidone,RISP1,021154,5.045803e+10,1mg tab,1,mg,1,tab,PO
701195,1497957,9895,100006,NaN,2108-04-13,2108-04-18,MAIN,magnesium oxide,Magnesium Oxide,Magnesium Oxide,MAGN400,001408,1.650022e+08,400mg tab,400,mg,1,tab,PO
701196,1497958,9895,100006,NaN,2108-04-13,2108-04-18,MAIN,albuterol neb soln,Albuterol Neb Soln,Albuterol Neb Soln,ALBU3H,005039,4.950207e+10,0.083%;3ml vl,1,neb,1,vial,IH
701197,1497959,9895,100006,NaN,2108-04-13,2108-04-18,MAIN,ipratropium bromide neb,Ipratropium Bromide Neb,Ipratropium Bromide Neb,IPRA2H,021700,4.950207e+10,2.5ml vial,1,neb,1,vial,IH
701198,1497962,9895,100006,NaN,2108-04-13,2108-04-18,MAIN,fluticasone propionate 110mcg,Fluticasone Propionate 110mcg,Fluticasone Propionate 110mcg,FLUT110H,021251,1.730494e+08,13gm inhaler,6,puff,0.06,inh,IH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703737,1497951,9895,100006,291788.0,2108-04-11,2108-04-18,MAIN,oxymetazoline hcl,Oxymetazoline HCl,Oxymetazoline HCl,OXYM15,008039,4.721596e+08,0.05% 15ml,1,spry,0.01666666666666666667,ml,NU
703738,1497947,9895,100006,291788.0,2108-04-11,2108-04-18,MAIN,risperidone,Risperidone,Risperidone,RISP05,042923,5.045803e+10,0.500mg tab,0.25,mg,0.5,tab,PO
703739,1497952,9895,100006,291788.0,2108-04-11,2108-04-18,MAIN,nystatin oral susp.,Nystatin Oral Susp.,Nystatin Oral Susp.,NYST5L,009537,5.096203e+10,500ku udcup,5,ml,1,udcup,PO
703740,1497954,9895,100006,291788.0,2108-04-12,2108-04-16,MAIN,sulfameth/trimethoprim ds,Sulfameth/Trimethoprim DS,Sulfameth/Trimethoprim DS,BACTDS,009396,9.042726e+08,ds tab,1,tab,1,tab,PO


## NDC = 0

In [9]:
df_prescriptions.NDC.value_counts()

0.000000e+00    477032
3.380017e+08     52057
4.096729e+08     46826
3.380049e+08     43961
3.380703e+08     41008
                 ...  
7.104172e+07         1
5.846847e+10         1
2.412001e+07         1
6.775331e+06         1
4.091181e+08         1
Name: NDC, Length: 4122, dtype: int64

In [10]:
_df_ndc_not_equal_0 = df_prescriptions[df_prescriptions.NDC != 0].copy()
_df_ndc_equal_0     = df_prescriptions[df_prescriptions.NDC == 0].copy()
_df_ndc_equal_0

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
3,2216265,6,107064,NaN,2175-06-11,2175-06-12,BASE,d5w,NaN,NaN,HEPBASE,NaN,0.0,heparin base,250,ml,250,ml,IV
10,2216267,6,107064,NaN,2175-06-12,2175-06-13,BASE,d5w,NaN,NaN,HEPBASE,NaN,0.0,heparin base,250,ml,250,ml,IV
16,2216268,6,107064,NaN,2175-06-13,2175-06-14,BASE,d5w,NaN,NaN,HEPBASE,NaN,0.0,heparin base,250,ml,250,ml,IV
23,2122384,13,143045,NaN,2167-01-08,2167-01-09,BASE,d5w,NaN,NaN,HEPBASE,NaN,0.0,heparin base,250,ml,250,ml,IV
27,2122374,13,143045,NaN,2167-01-08,2167-01-09,MAIN,insulin,Insulin,Insulin,INSULIN,001723,0.0,dummy pkg for poe pump and pha ss ordering,0,unit,0,vial,SC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4156417,3488589,98883,144680,235104.0,2102-01-30,2102-01-31,BASE,bag,NaN,NaN,BAG50,NaN,0.0,50 ml bag,100,ml,2,bag,IV
4156421,3488587,98883,144680,235104.0,2102-01-30,2102-01-31,BASE,sw,NaN,NaN,KCLBASE,NaN,0.0,100ml bag,100,ml,100,ml,IV
4156422,3488586,98883,144680,235104.0,2102-01-30,2102-01-31,BASE,sw,NaN,NaN,KCLBASE,NaN,0.0,100ml bag,100,ml,100,ml,IV
4156423,3488040,98883,144680,235104.0,2102-01-30,2102-02-01,MAIN,insulin,Insulin,Insulin - Sliding Scale,INSULIN,001723,0.0,dummy package for sliding scale,0,unit,0,vial,SC


In [11]:
_df_ndc_equal_0.DRUG.value_counts()

sodium chloride 0.9%  flush     79110
iso-osmotic dextrose            69713
insulin                         67200
sw                              60361
d5w                             37456
                                ...  
pantoprazole                        1
sodium chloride                     1
tilarginine or placebo *ind*        1
viread                              1
insulin human regular               1
Name: DRUG, Length: 174, dtype: int64

In [12]:
_df_ndc_not_equal_0.NDC.min(), \
_df_ndc_not_equal_0.NDC.max()

(2050101.0, 87701089415.0)

Alternative solution: remapping these drug using number < 2050101.

In [13]:
remap_4_ndc_equal_0 = _df_ndc_equal_0.DRUG.value_counts()
remap_4_ndc_equal_0 = pd.Series(
    index=remap_4_ndc_equal_0.index, 
    data=range(1, len(remap_4_ndc_equal_0)+1)
)
remap_4_ndc_equal_0

sodium chloride 0.9%  flush       1
iso-osmotic dextrose              2
insulin                           3
sw                                4
d5w                               5
                               ... 
pantoprazole                    170
sodium chloride                 171
tilarginine or placebo *ind*    172
viread                          173
insulin human regular           174
Length: 174, dtype: int64

In [ ]:
_df_ndc_equal_0['NDC'] = _df_ndc_equal_0['DRUG'].map(remap_4_ndc_equal_0)
_df_ndc_equal_0['NDC'].value_counts()

In [15]:
df_prescriptions = pd.concat([_df_ndc_equal_0, _df_ndc_not_equal_0])

## Druation issuse

The duration of some drugs may longer than one day.

The alternative for dealing this issuss may be simplily repeating those entries, and re-map their STARTDATE and ENDDATE.

In [16]:
grouped_by_hadmid = df_prescriptions.groupby("HADM_ID")

In [17]:
HADM_ID = 100001
grouped_by_hadmid.get_group(HADM_ID).sort_values(by=["STARTDATE", "ENDDATE"])

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
3026450,3967375,58526,100001,275225.0,2117-09-11,2117-09-12,BASE,potassium chl 20 meq / 1000 ml d5ns,NaN,NaN,KCL20/1000D5NS,011988,3.380803e+08,1000 ml bag,1000,ml,1,bag,IV
3026451,3967376,58526,100001,275225.0,2117-09-11,2117-09-12,BASE,0.9% sodium chloride,NaN,NaN,NS500,001210,3.380049e+08,500ml bag,500,ml,500,ml,IV
3026452,3965948,58526,100001,275225.0,2117-09-11,2117-09-12,MAIN,metoprolol tartrate,Metoprolol Tartrate,Metoprolol Tartrate,METO5I,019808,5.539003e+10,5mg/5ml vial,5,mg,1,vial,IV
3026453,3965947,58526,100001,275225.0,2117-09-11,2117-09-12,MAIN,promethazine,Promethazine,Promethazine,PROM25I,003870,6.410928e+08,25mg/ml amp,6.25-12.5,mg,0.25-0.5,amp,IV
3026454,3967374,58526,100001,275225.0,2117-09-11,2117-09-13,BASE,0.9% sodium chloride,NaN,NaN,NS100,001210,3.380049e+08,100ml bag,100,ml,100,ml,IV DRIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3023196,3966722,58526,100001,275225.0,2117-09-15,2117-09-17,MAIN,sodium chloride 0.9% flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,1.000000e+00,syringe,3,ml,0.6,syr,IV
3023194,3966725,58526,100001,275225.0,2117-09-15,2117-09-17,MAIN,insulin,Insulin,Insulin Glargine,GLAR100I,047780,8.822203e+07,100 units / ml - 10 ml vial,20,unit,0.02,vial,SC
3023197,3966724,58526,100001,275225.0,2117-09-15,2117-09-17,MAIN,insulin,Insulin,Insulin Glargine,GLAR100I,047780,8.822203e+07,100 units / ml - 10 ml vial,10,unit,0.01,vial,SC
3023198,3966728,58526,100001,275225.0,2117-09-16,2117-09-17,MAIN,metoprolol tartrate,Metoprolol Tartrate,Metoprolol Tartrate,METO25,050631,5.107903e+10,25mg tablet,25,mg,1,tab,PO/NG


In [18]:
temp_df = grouped_by_hadmid.get_group(HADM_ID)
mask = (temp_df.ENDDATE - temp_df.STARTDATE) > timedelta(hours=24)
temp_df.loc[mask]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
3023195,3966723,58526,100001,275225.0,2117-09-15,2117-09-17,MAIN,insulin,Insulin,Insulin - Sliding Scale,INSULIN,027413,3.000000e+00,dummy package for sliding scale,0,unit,0,vial,SC
3023196,3966722,58526,100001,275225.0,2117-09-15,2117-09-17,MAIN,sodium chloride 0.9% flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,1.000000e+00,syringe,3,ml,0.6,syr,IV
3024828,3965963,58526,100001,275225.0,2117-09-12,2117-09-14,BASE,vial,NaN,NaN,VIAL,NaN,7.000000e+00,send vial,1,vial,1,vial,IV
3024829,3965955,58526,100001,275225.0,2117-09-12,2117-09-15,BASE,bag,NaN,NaN,BAG50,NaN,6.000000e+00,50 ml bag,50,ml,1,bag,IV
3024831,3965957,58526,100001,275225.0,2117-09-12,2117-09-15,BASE,bag,NaN,NaN,BAG50,NaN,6.000000e+00,50 ml bag,50,ml,1,bag,IV
3024833,3965956,58526,100001,275225.0,2117-09-12,2117-09-15,BASE,bag,NaN,NaN,BAG50,NaN,6.000000e+00,50 ml bag,50,ml,1,bag,IV
3026458,3965937,58526,100001,275225.0,2117-09-11,2117-09-17,MAIN,sodium chloride 0.9% flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,1.000000e+00,syringe,3,ml,0.6,syr,IV
3023189,3966714,58526,100001,275225.0,2117-09-14,2117-09-17,MAIN,glucagon,Glucagon,Glucagon,GLUC1I,041660,5.539000e+10,1mg vial,1,mg,1,vial,IM
3023190,3966713,58526,100001,275225.0,2117-09-14,2117-09-17,MAIN,dextrose 50%,Dextrose 50%,Dextrose 50%,DEX50SY,001989,4.094902e+08,50ml syringe,12.5,gm,0.5,syr,IV
3023194,3966725,58526,100001,275225.0,2117-09-15,2117-09-17,MAIN,insulin,Insulin,Insulin Glargine,GLAR100I,047780,8.822203e+07,100 units / ml - 10 ml vial,20,unit,0.02,vial,SC


In [19]:
temp_df.loc[mask].iloc[0].STARTDATE, temp_df.loc[mask].iloc[0].ENDDATE

(Timestamp('2117-09-15 00:00:00'), Timestamp('2117-09-17 00:00:00'))

In [20]:
temp_df.loc[mask].iloc[0].ENDDATE - temp_df.loc[mask].iloc[0].STARTDATE

Timedelta('2 days 00:00:00')

In [21]:
(temp_df.loc[mask].iloc[0].ENDDATE - temp_df.loc[mask].iloc[0].STARTDATE) / timedelta(hours=24)

2.0

In [22]:
temp_df_duration_longer_than_1 = temp_df.loc[mask].reindex(
    temp_df.loc[mask].index.repeat(
        (temp_df.loc[mask].ENDDATE - temp_df.loc[mask].STARTDATE) / timedelta(hours=24)
    )
).reset_index(drop=True)
temp_df_duration_longer_than_1

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,3966723,58526,100001,275225.0,2117-09-15,2117-09-17,MAIN,insulin,Insulin,Insulin - Sliding Scale,INSULIN,027413,3.0,dummy package for sliding scale,0,unit,0,vial,SC
1,3966723,58526,100001,275225.0,2117-09-15,2117-09-17,MAIN,insulin,Insulin,Insulin - Sliding Scale,INSULIN,027413,3.0,dummy package for sliding scale,0,unit,0,vial,SC
2,3966722,58526,100001,275225.0,2117-09-15,2117-09-17,MAIN,sodium chloride 0.9% flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,1.0,syringe,3,ml,0.6,syr,IV
3,3966722,58526,100001,275225.0,2117-09-15,2117-09-17,MAIN,sodium chloride 0.9% flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,1.0,syringe,3,ml,0.6,syr,IV
4,3965963,58526,100001,275225.0,2117-09-12,2117-09-14,BASE,vial,NaN,NaN,VIAL,NaN,7.0,send vial,1,vial,1,vial,IV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,3965941,58526,100001,275225.0,2117-09-11,2117-09-17,MAIN,gabapentin,Gabapentin,Gabapentin,GABA300,021414,172438210.0,300mg capsule,300,mg,1,cap,PO/NG
105,3965941,58526,100001,275225.0,2117-09-11,2117-09-17,MAIN,gabapentin,Gabapentin,Gabapentin,GABA300,021414,172438210.0,300mg capsule,300,mg,1,cap,PO/NG
106,3965941,58526,100001,275225.0,2117-09-11,2117-09-17,MAIN,gabapentin,Gabapentin,Gabapentin,GABA300,021414,172438210.0,300mg capsule,300,mg,1,cap,PO/NG
107,3965941,58526,100001,275225.0,2117-09-11,2117-09-17,MAIN,gabapentin,Gabapentin,Gabapentin,GABA300,021414,172438210.0,300mg capsule,300,mg,1,cap,PO/NG


In [23]:
gb_duration_longer_than_1 = temp_df_duration_longer_than_1.groupby("ROW_ID")
gb_duration_longer_than_1.get_group(3966723)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,3966723,58526,100001,275225.0,2117-09-15,2117-09-17,MAIN,insulin,Insulin,Insulin - Sliding Scale,INSULIN,027413,3.0,dummy package for sliding scale,0,unit,0,vial,SC
1,3966723,58526,100001,275225.0,2117-09-15,2117-09-17,MAIN,insulin,Insulin,Insulin - Sliding Scale,INSULIN,027413,3.0,dummy package for sliding scale,0,unit,0,vial,SC


In [24]:
def modify_timestamp_4_duration_longer_than_1(df):
    num_duration = len(df)
    dfx = df.copy()
    interval_1day = timedelta(hours=24)
    
    curr_idx = 0
    curr_st = dfx.iloc[curr_idx].STARTDATE
    curr_et = curr_st + interval_1day
    
    while curr_idx < num_duration:
        dfx["STARTDATE"].iloc[curr_idx] = curr_st
        dfx["ENDDATE"].iloc[curr_idx] = curr_et
        
        curr_idx += 1
        curr_st += interval_1day
        curr_et += interval_1day
    
    return dfx

modify_timestamp_4_duration_longer_than_1(gb_duration_longer_than_1.get_group(3966723))

/data/data2/041/envs/LERS2/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,3966723,58526,100001,275225.0,2117-09-15,2117-09-16,MAIN,insulin,Insulin,Insulin - Sliding Scale,INSULIN,027413,3.0,dummy package for sliding scale,0,unit,0,vial,SC
1,3966723,58526,100001,275225.0,2117-09-16,2117-09-17,MAIN,insulin,Insulin,Insulin - Sliding Scale,INSULIN,027413,3.0,dummy package for sliding scale,0,unit,0,vial,SC


In [25]:
gb_duration_longer_than_1.apply(modify_timestamp_4_duration_longer_than_1)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,3966723,58526,100001,275225.0,2117-09-15,2117-09-16,MAIN,insulin,Insulin,Insulin - Sliding Scale,INSULIN,027413,3.0,dummy package for sliding scale,0,unit,0,vial,SC
1,3966723,58526,100001,275225.0,2117-09-16,2117-09-17,MAIN,insulin,Insulin,Insulin - Sliding Scale,INSULIN,027413,3.0,dummy package for sliding scale,0,unit,0,vial,SC
2,3966722,58526,100001,275225.0,2117-09-15,2117-09-16,MAIN,sodium chloride 0.9% flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,1.0,syringe,3,ml,0.6,syr,IV
3,3966722,58526,100001,275225.0,2117-09-16,2117-09-17,MAIN,sodium chloride 0.9% flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,1.0,syringe,3,ml,0.6,syr,IV
4,3965963,58526,100001,275225.0,2117-09-12,2117-09-13,BASE,vial,NaN,NaN,VIAL,NaN,7.0,send vial,1,vial,1,vial,IV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,3965941,58526,100001,275225.0,2117-09-12,2117-09-13,MAIN,gabapentin,Gabapentin,Gabapentin,GABA300,021414,172438210.0,300mg capsule,300,mg,1,cap,PO/NG
105,3965941,58526,100001,275225.0,2117-09-13,2117-09-14,MAIN,gabapentin,Gabapentin,Gabapentin,GABA300,021414,172438210.0,300mg capsule,300,mg,1,cap,PO/NG
106,3965941,58526,100001,275225.0,2117-09-14,2117-09-15,MAIN,gabapentin,Gabapentin,Gabapentin,GABA300,021414,172438210.0,300mg capsule,300,mg,1,cap,PO/NG
107,3965941,58526,100001,275225.0,2117-09-15,2117-09-16,MAIN,gabapentin,Gabapentin,Gabapentin,GABA300,021414,172438210.0,300mg capsule,300,mg,1,cap,PO/NG


In [ ]:
interval_1day = timedelta(hours=24)

mask = (df_prescriptions.ENDDATE - df_prescriptions.STARTDATE) > interval_1day
df_duration_longer_than_1 = df_prescriptions.loc[mask].copy()
df_duration_equal_to_1day = df_prescriptions.loc[~mask].copy()

df_duration_longer_than_1 = df_duration_longer_than_1.reindex(
    df_duration_longer_than_1.index.repeat(
        (df_duration_longer_than_1.ENDDATE - df_duration_longer_than_1.STARTDATE) / interval_1day
    )
).reset_index(drop=True)

gb_duration_longer_than_1 = df_duration_longer_than_1.groupby("ROW_ID")

df_prescriptions_duration_solved = pd.DataFrame()
list_df_prescriptions_duration_solved = []
for key, df in tqdm(gb_duration_longer_than_1):
    list_df_prescriptions_duration_solved.append(modify_timestamp_4_duration_longer_than_1(df))
    if len(list_df_prescriptions_duration_solved) % 10000 == 0:
        list_df_prescriptions_duration_solved.append(df_prescriptions_duration_solved)
        df_prescriptions_duration_solved = pd.concat(list_df_prescriptions_duration_solved)
        list_df_prescriptions_duration_solved = []
        
list_df_prescriptions_duration_solved.append(df_prescriptions_duration_solved)
df_prescriptions_duration_solved = pd.concat(list_df_prescriptions_duration_solved)
df_prescriptions_duration_solved = pd.concat([df_prescriptions_duration_solved, df_duration_equal_to_1day]).reset_index(drop=True)

df_prescriptions_duration_solved.to_csv(os.path.join(constant.PATH_MIMIC_III_ETL_OUTPUT, "PRESCRIPTIONS_DURATION_SOLVED.csv.gz"))

In [ ]:
df_prescriptions = df_prescriptions_duration_solved

## Multi strength / drug feature

In [98]:
df_prescriptions.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,0,4132229,66412,187038,278897.0,2155-11-04,2155-11-05,MAIN,sodium chloride 0.9% flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,2.000000e+00,syringe,10,ml,2,syr,IV
1,1,4132229,66412,187038,278897.0,2155-11-05,2155-11-06,MAIN,sodium chloride 0.9% flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,2.000000e+00,syringe,10,ml,2,syr,IV
2,2,4132229,66412,187038,278897.0,2155-11-06,2155-11-07,MAIN,sodium chloride 0.9% flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,2.000000e+00,syringe,10,ml,2,syr,IV
3,3,4132231,66412,187038,278897.0,2155-11-04,2155-11-05,MAIN,propofol,Propofol,Propofol,PROP100IG,16796.0,6.332303e+10,1000mg/100ml vial,1000,mg,1,vial,IV DRIP
4,4,4132231,66412,187038,278897.0,2155-11-05,2155-11-06,MAIN,propofol,Propofol,Propofol,PROP100IG,16796.0,6.332303e+10,1000mg/100ml vial,1000,mg,1,vial,IV DRIP


### single drug may have multiple strength

In [43]:
list_multi_strength_drug_ndc = []

for ndc in tqdm(df_prescriptions.NDC.unique()):
    if len(df_prescriptions[df_prescriptions.NDC == ndc].PROD_STRENGTH.value_counts()) > 1:
        list_multi_strength_drug_ndc.append(ndc)
        
len(list_multi_strength_drug_ndc)

  0%|          | 0/4392 [00:00<?, ?it/s]

1538

In [44]:
df_prescriptions[df_prescriptions.NDC == 469060773].PROD_STRENGTH.value_counts()

0.5mg Capsule             2616
Check with MD for Dose     258
0.5 mg Capsule              73
Name: PROD_STRENGTH, dtype: int64

In [45]:
_temp_df = df_prescriptions[df_prescriptions.NDC == 469060773]
_temp_df[_temp_df.PROD_STRENGTH == "Check with MD for Dose"]

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
553340,553340,3929991,78342,171372,NaN,2199-11-03,2199-11-04,MAIN,tacrolimus,Tacrolimus,Tacrolimus,TACR0,41832.0,469060773.0,Check with MD for Dose,1,dose,1,dose,PO
553341,553341,3929991,78342,171372,NaN,2199-11-04,2199-11-05,MAIN,tacrolimus,Tacrolimus,Tacrolimus,TACR0,41832.0,469060773.0,Check with MD for Dose,1,dose,1,dose,PO
699642,699642,3847434,58010,185994,214987.0,2109-03-18,2109-03-19,MAIN,tacrolimus,Tacrolimus,Tacrolimus,TACR0,41832.0,469060773.0,Check with MD for Dose,1,dose,1,dose,PO
699643,699643,3847434,58010,185994,214987.0,2109-03-19,2109-03-20,MAIN,tacrolimus,Tacrolimus,Tacrolimus,TACR0,41832.0,469060773.0,Check with MD for Dose,1,dose,1,dose,PO
699644,699644,3847434,58010,185994,214987.0,2109-03-20,2109-03-21,MAIN,tacrolimus,Tacrolimus,Tacrolimus,TACR0,41832.0,469060773.0,Check with MD for Dose,1,dose,1,dose,PO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12593862,12593862,4020803,87692,196705,288862.0,2151-02-02,2151-02-03,MAIN,tacrolimus,Tacrolimus,Tacrolimus,TACR0,41832,469060773.0,Check with MD for Dose,0,mg,1,dose,PO
12704945,12704945,1495063,93541,160567,NaN,2107-03-13,2107-03-14,MAIN,tacrolimus,Tacrolimus,Tacrolimus,TACR0,41832,469060773.0,Check with MD for Dose,0,mg,1,dose,PO
12704946,12704946,1495062,93541,160567,NaN,2107-03-13,2107-03-14,MAIN,tacrolimus,Tacrolimus,Tacrolimus,TACR0,41832,469060773.0,Check with MD for Dose,1,dose,1,dose,PO
12750713,12750713,676411,98851,105014,NaN,2155-04-22,2155-04-23,MAIN,tacrolimus,Tacrolimus,Tacrolimus,TACR0,41832,469060773.0,Check with MD for Dose,0,mg,1,dose,PO


The dose val may also be multiple for single drug.

In [20]:
df_prescriptions[df_prescriptions.NDC == 641092825].DOSE_VAL_RX.value_counts()

25           1940
12.5         1881
6.25         1599
6.25-12.5     563
12.5-25       215
3.125          23
25-50          21
6.25-25         8
12.5-25.0       7
6.25 - 12       7
50              5
12.5-37.5       2
6.25-12         1
3.12            1
Name: DOSE_VAL_RX, dtype: int64

### NCV converting

In [157]:
ncv_cols_global = ["DRUG_TYPE", "DOSE_UNIT_RX", "FORM_UNIT_DISP", "ROUTE"]

for c in ncv_cols_global:
    m = df_prescriptions[c].value_counts()
    m = pd.Series(index=m.index, data=range(1, len(m)+1))
    df_prescriptions[c] = df_prescriptions[c].map(m)
    
values_fillna = {}
for c in ncv_cols_global:
    values_fillna[c] = 0  # use 0 to fill na
df_prescriptions.fillna(value=values_fillna, inplace=True)
df_prescriptions.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,0,4132229,66412,187038,278897.0,2155-11-04,2155-11-05,1,sodium chloride 0.9% flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,2.000000e+00,syringe,10,2.0,2,5.0,1
1,1,4132229,66412,187038,278897.0,2155-11-05,2155-11-06,1,sodium chloride 0.9% flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,2.000000e+00,syringe,10,2.0,2,5.0,1
2,2,4132229,66412,187038,278897.0,2155-11-06,2155-11-07,1,sodium chloride 0.9% flush,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,2.000000e+00,syringe,10,2.0,2,5.0,1
3,3,4132231,66412,187038,278897.0,2155-11-04,2155-11-05,1,propofol,Propofol,Propofol,PROP100IG,16796.0,6.332303e+10,1000mg/100ml vial,1000,1.0,1,3.0,5
4,4,4132231,66412,187038,278897.0,2155-11-05,2155-11-06,1,propofol,Propofol,Propofol,PROP100IG,16796.0,6.332303e+10,1000mg/100ml vial,1000,1.0,1,3.0,5


In [158]:
# below cols are special with each NDC
ncv_cols = ["PROD_STRENGTH", "DOSE_VAL_RX", "FORM_VAL_DISP"]

def convert_ncv(df_curr_ndc):
    dfx = df_curr_ndc.copy()
    
    for col in ncv_cols:
        m = df_curr_ndc[col].value_counts()
        m = pd.Series(index=m.index, data=range(1, len(m)+1))
        dfx[col] = dfx[col].map(m)
    
    return dfx

gb_ndc = df_prescriptions.groupby("NDC")
convert_ncv(gb_ndc.get_group(641092825))

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
1799,1799,4133286,74252,159132,290918.0,2154-08-20,2154-08-21,1,promethazine,Promethazine,Promethazine,PROM25I,3870.0,641092825.0,1,2,1.0,2,13.0,1
1800,1800,4133286,74252,159132,290918.0,2154-08-21,2154-08-22,1,promethazine,Promethazine,Promethazine,PROM25I,3870.0,641092825.0,1,2,1.0,2,13.0,1
1801,1801,4133286,74252,159132,290918.0,2154-08-22,2154-08-23,1,promethazine,Promethazine,Promethazine,PROM25I,3870.0,641092825.0,1,2,1.0,2,13.0,1
1802,1802,4133286,74252,159132,290918.0,2154-08-23,2154-08-24,1,promethazine,Promethazine,Promethazine,PROM25I,3870.0,641092825.0,1,2,1.0,2,13.0,1
12571,12571,4137474,63383,191404,NaN,2194-08-02,2194-08-03,1,promethazine,Promethazine,Promethazine,PROM25I,3870.0,641092825.0,1,3,1.0,3,13.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12761289,12761289,3323675,99287,182660,276767.0,2135-12-06,2135-12-06,1,promethazine hcl,Promethazine HCl,Promethazine,PROM25I,3870,641092825.0,1,4,1.0,4,13.0,1
12763975,12763975,2027438,99384,147442,201477.0,2117-11-30,2117-11-30,1,promethazine hcl,Promethazine HCl,Promethazine,PROM25I,3870,641092825.0,1,4,1.0,4,13.0,1
12770049,12770049,2703809,99141,144869,251582.0,2105-12-05,2105-12-05,1,promethazine,Promethazine,Promethazine,PROM25I,3870,641092825.0,1,3,1.0,3,13.0,1
12770050,12770050,2703810,99141,144869,251582.0,2105-12-05,2105-12-05,1,promethazine,Promethazine,Promethazine,PROM25I,3870,641092825.0,1,5,1.0,5,13.0,1


In [159]:
gb_ndc = df_prescriptions.groupby("NDC")
ncv_cols = ["PROD_STRENGTH", "DOSE_VAL_RX", "FORM_VAL_DISP"]

list_dfx = []
dfxs_ncv_converted = pd.DataFrame()
for k, df in tqdm(gb_ndc):
    list_dfx.append(convert_ncv(df))
    if len(list_dfx) % 100 == 0:
        list_dfx.append(dfxs_ncv_converted)
        dfxs_ncv_converted = pd.concat(list_dfx)
        list_dfx = []

list_dfx.append(dfxs_ncv_converted)
dfxs_ncv_converted = pd.concat(list_dfx)

values_fillna = {}
for c in ncv_cols:
    values_fillna[c] = 0  # use 0 to fill na
dfxs_ncv_converted.fillna(value=values_fillna, inplace=True)

dfxs_ncv_converted.head()

  0%|          | 0/4392 [00:00<?, ?it/s]

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
1712127,1712127,3397704,58848,191054,224045.0,2193-04-23,2193-04-24,1,acebutolol hcl,Acebutolol HCl,Acebutolol HCl,ACEB200,5145.0,6.728642e+10,1,1,1.0,1,6.0,2
1712128,1712128,3397704,58848,191054,224045.0,2193-04-24,2193-04-25,1,acebutolol hcl,Acebutolol HCl,Acebutolol HCl,ACEB200,5145.0,6.728642e+10,1,1,1.0,1,6.0,2
2320676,2320676,3117724,29906,181939,297744.0,2134-10-05,2134-10-06,1,acebutolol hcl,Acebutolol HCl,Acebutolol HCl,ACEB200,5145.0,6.728642e+10,1,1,1.0,1,6.0,2
2320677,2320677,3117724,29906,181939,297744.0,2134-10-06,2134-10-07,1,acebutolol hcl,Acebutolol HCl,Acebutolol HCl,ACEB200,5145.0,6.728642e+10,1,1,1.0,1,6.0,2
2451986,2451986,3075384,32050,150375,NaN,2122-11-08,2122-11-09,1,acebutolol hcl,Acebutolol HCl,Acebutolol HCl,ACEB200,5145.0,6.728642e+10,1,1,1.0,1,6.0,2


In [160]:
df_prescriptions = dfxs_ncv_converted
df_prescriptions.describe()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DRUG_TYPE,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
count,1.276649e+07,1.276649e+07,1.276649e+07,1.276649e+07,8.053513e+06,1.276649e+07,1.276649e+07,1.276649e+07,1.276649e+07,1.276649e+07,1.276649e+07,1.276649e+07,1.276649e+07
mean,6.383558e+06,2.051374e+06,3.681009e+04,1.502249e+05,2.498575e+05,1.170961e+00,1.330479e+10,1.143803e+00,1.848838e+00,3.132042e+00,1.864409e+00,3.771045e+00,2.923949e+00
std,3.685853e+06,1.178155e+06,2.890910e+04,2.890299e+04,2.862168e+04,3.811451e-01,2.365551e+10,4.736910e-01,4.529475e+00,3.993151e+00,4.501737e+00,3.567014e+00,3.433418e+00
min,0.000000e+00,1.000000e+00,2.000000e+00,1.000010e+05,2.000010e+05,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00
25%,3.191623e+06,1.054684e+06,1.345100e+04,1.252350e+05,2.253320e+05,1.000000e+00,5.484862e+07,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00
50%,6.383246e+06,2.014872e+06,2.686000e+04,1.504890e+05,2.499040e+05,1.000000e+00,3.380550e+08,1.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,3.000000e+00,2.000000e+00
75%,9.574870e+06,3.048142e+06,5.988900e+04,1.751580e+05,2.742530e+05,1.000000e+00,1.001900e+10,1.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00,5.000000e+00,3.000000e+00
max,1.277097e+07,4.156450e+06,9.999900e+04,1.999990e+05,2.999990e+05,3.000000e+00,8.770109e+10,1.400000e+01,4.260000e+02,9.100000e+01,4.210000e+02,5.400000e+01,7.700000e+01


## Adding timestep

In [173]:
HADM_ID = 100006
df_admissions[df_admissions.HADM_ID == HADM_ID]

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
9046,12108,9895,100006,2108-04-06 15:49:00,2108-04-18 17:18:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2108-04-06 11:39:00,2108-04-06 17:56:00,COPD FLARE,0,1


In [162]:
df_admissions.ADMITTIME.isnull().sum()

0

In [174]:
def add_timestep_per_hadmid(df_grouped_by_hadmid: pd.DataFrame):
    interval_hour = 24  # chosen interval
    df_grouped_by_hadmid = df_grouped_by_hadmid.sort_values(by=["STARTDATE", "ENDDATE"])

    hadm_id = df_grouped_by_hadmid.HADM_ID.unique()[0]
    st = df_admissions[df_admissions.HADM_ID == hadm_id].ADMITTIME.iloc[0]
    et = df_grouped_by_hadmid[df_grouped_by_hadmid.HADM_ID == hadm_id].ENDDATE.iloc[-1]
    # The end time of the prescription may be after the discharge time

    st = datetime.strptime(f"{st.year}-{st.month}-{st.day} {st.hour // interval_hour * interval_hour:2}:00:00", "%Y-%m-%d %H:%M:%S")
    et = datetime.strptime(f"{et.year}-{et.month}-{et.day} {(((et.hour // interval_hour) + 1) * interval_hour) - 1:2}:59:59", "%Y-%m-%d %H:%M:%S")

    interval = timedelta(hours=interval_hour)

    dfx = df_grouped_by_hadmid.copy()
    # filter out records whose `STARTDATE` are earlier than `ADMITTIME` which will cause `nan` issues!!!
    dfx = dfx[dfx.STARTDATE >= st]
    dfx.insert(len(dfx.columns), "TIMESTEP", np.NaN)

    timestep = 0
    while st < et:
        mask = (st <= dfx.STARTDATE) & (dfx.STARTDATE <= st + interval)
        if len(dfx.loc[mask]) > 0:
            dfx.loc[mask, 'TIMESTEP'] = timestep

        timestep += 1
        st += interval

    return dfx

grouped_by_hadmid = df_prescriptions.groupby("HADM_ID")

HADM_ID = 100006
add_timestep_per_hadmid(grouped_by_hadmid.get_group(HADM_ID))

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,...,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,TIMESTEP
6608036,6608036,1497260,9895,100006,NaN,2108-04-06,2108-04-07,1,pantoprazole,Pantoprazole,...,PANT40,27462.0,8.084199e+06,2,1,1.0,1,1.0,2,0.0
6608024,6608024,1497259,9895,100006,NaN,2108-04-06,2108-04-07,1,warfarin,Warfarin,...,WARF5,6562.0,5.601728e+07,2,1,1.0,1,1.0,2,0.0
6608048,6608048,1497261,9895,100006,NaN,2108-04-06,2108-04-07,1,insulin,Insulin,...,INSULIN,1723.0,3.000000e+00,2,1,3.0,2,3.0,4,0.0
6607992,6607992,1497255,9895,100006,NaN,2108-04-06,2108-04-07,1,albuterol neb soln,Albuterol Neb Soln,...,ALBU3H,5039.0,4.950207e+10,2,1,7.0,1,3.0,6,0.0
6608008,6608008,1497257,9895,100006,NaN,2108-04-06,2108-04-07,1,azithromycin,Azithromycin,...,ZITHR250,16630.0,6.930609e+07,3,2,1.0,2,6.0,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6610268,6610268,1497951,9895,100006,291788.0,2108-04-17,2108-04-18,1,oxymetazoline hcl,Oxymetazoline HCl,...,OXYM15,8039.0,4.721596e+08,1,1,17.0,1,2.0,16,11.0
6608019,6608019,1497257,9895,100006,NaN,2108-04-17,2108-04-18,1,azithromycin,Azithromycin,...,ZITHR250,16630.0,6.930609e+07,3,2,1.0,2,6.0,2,11.0
10384307,10384307,1497252,9895,100006,NaN,2108-04-17,2108-04-18,2,syringe (chemo),NaN,...,CHEMOSYR,NaN,3.800000e+01,1,1,2.0,2,2.0,1,11.0
10995642,10995642,1497964,9895,100006,NaN,2108-04-21,2108-04-18,1,prednisone,Prednisone,...,PRED10,6749,5.487252e+07,1,2,1.0,2,1.0,2,NaN


In [164]:
df_prescriptions_preprocessed = pd.DataFrame()  # blank DataFrame
list_df_prescriptions_preprocessed = []
for k, df in tqdm(grouped_by_hadmid):
    list_df_prescriptions_preprocessed.append(add_timestep_per_hadmid(df))
    if len(list_df_prescriptions_preprocessed) % 1000 == 0:
        list_df_prescriptions_preprocessed.append(df_prescriptions_preprocessed)
        df_prescriptions_preprocessed = pd.concat(list_df_prescriptions_preprocessed)
        list_df_prescriptions_preprocessed = []
    
list_df_prescriptions_preprocessed.append(df_prescriptions_preprocessed)
df_prescriptions_preprocessed = pd.concat(list_df_prescriptions_preprocessed)

  0%|          | 0/50208 [00:00<?, ?it/s]

In [114]:
df_prescriptions_preprocessed.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,...,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,TIMESTEP
3177527,3177527,2789566,4036,199606,290402.0,2195-08-07,2195-08-08,1,neo*iv*gentamicin,NaN,...,GENT10I,9298.0,6.332302e+10,1.0,26,1.0,31,2.0,1.0,1.0
3177529,3177529,2789567,4036,199606,290402.0,2195-08-07,2195-08-08,1,neo*iv*ampicillin sodium,NaN,...,NAMP500I,8937.0,6.332304e+10,1.0,14,1.0,14,3.0,1.0,1.0
3177523,3177523,2789564,4036,199606,290402.0,2195-08-07,2195-08-08,2,syringe (neonatal) *d5w*,NaN,...,NEOSYRD5W,NaN,1.700000e+01,1.0,1,2.0,1,2.0,1.0,1.0
3177525,3177525,2789565,4036,199606,290402.0,2195-08-07,2195-08-08,2,send 500mg vial,NaN,...,AMPVL,NaN,2.000000e+01,1.0,1,12.0,1,3.0,1.0,1.0
3177528,3177528,2789566,4036,199606,290402.0,2195-08-08,2195-08-09,1,neo*iv*gentamicin,NaN,...,GENT10I,9298.0,6.332302e+10,1.0,26,1.0,31,2.0,1.0,2.0


## Checking of edges repeating issuse 

In [115]:
gb_hadmid = df_prescriptions_preprocessed.groupby("HADM_ID")
gb_hadmid.get_group(100001)

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,...,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,TIMESTEP
414991,414991,3965945,58526,100001,275225.0,2117-09-11,2117-09-12,1,heparin,Heparin,...,HEPA5I,6549.0,6.332303e+10,1.0,1,3.0,1,2.0,4.0,0.0
12186882,12186882,3967375,58526,100001,275225.0,2117-09-11,2117-09-12,2,potassium chl 20 meq / 1000 ml d5ns,NaN,...,KCL20/1000D5NS,11988,3.380803e+08,1.0,1,2.0,1,4.0,1.0,0.0
12186874,12186874,3967370,58526,100001,275225.0,2117-09-11,2117-09-11,2,0.45% sodium chloride,NaN,...,NS451000,1209,3.380043e+08,1.0,1,2.0,1,2.0,1.0,0.0
414973,414973,3965941,58526,100001,275225.0,2117-09-11,2117-09-12,1,gabapentin,Gabapentin,...,GABA300,21414.0,1.724382e+08,1.0,1,1.0,1,6.0,3.0,0.0
12186884,12186884,3965948,58526,100001,275225.0,2117-09-11,2117-09-12,1,metoprolol tartrate,Metoprolol Tartrate,...,METO5I,19808,5.539003e+10,1.0,1,1.0,1,3.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414990,414990,3965943,58526,100001,275225.0,2117-09-16,2117-09-17,1,oxycodone (immediate release),OxycoDONE (Immediate Release),...,OXYC5,4225.0,4.060553e+08,1.0,1,1.0,1,1.0,3.0,5.0
416286,416286,3966703,58526,100001,275225.0,2117-09-16,2117-09-17,1,promethazine,Promethazine,...,PROM25I,3870.0,6.410928e+08,1.0,1,1.0,1,13.0,1.0,5.0
414978,414978,3965941,58526,100001,275225.0,2117-09-16,2117-09-17,1,gabapentin,Gabapentin,...,GABA300,21414.0,1.724382e+08,1.0,1,1.0,1,6.0,3.0,5.0
12185199,12185199,3967369,58526,100001,NaN,2117-09-17,2117-09-17,1,midodrine,Midodrine,...,MIDO25,17118,5.409200e+10,1.0,2,1.0,2,1.0,2.0,6.0


In [117]:
_df_curr_hadm_id = gb_hadmid.get_group(100001)
_df_curr_hadm_id.sort_values(by=["STARTDATE", "ENDDATE"])
max_timestep = _df_curr_hadm_id.TIMESTEP.max()
_df_curr_hadmid_curr_timestep = _df_curr_hadm_id[_df_curr_hadm_id.TIMESTEP == 1]
sr_itemid_value_counts = _df_curr_hadmid_curr_timestep.NDC.value_counts()
sr_itemid_repeat = sr_itemid_value_counts[sr_itemid_value_counts > 1]

sr_itemid_repeat

4.091985e+08    4
4.096729e+08    3
6.000000e+00    3
3.380117e+08    3
3.380049e+08    2
3.380085e+08    2
5.539003e+10    2
3.380089e+08    2
1.747809e+10    2
Name: NDC, dtype: int64

In [135]:
_df_curr_hadmid_curr_timestep[
    _df_curr_hadmid_curr_timestep.NDC == sr_itemid_repeat.index[-1]
].sort_values(by=["STARTDATE", "ENDDATE"])

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,...,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,TIMESTEP
12186081,12186081,3966705,58526,100001,275225.0,2117-09-12,2117-09-13,1,hydralazine,HydrALAzine,...,HYDZ20I,283,1.747809e+10,1.0,1,1.0,1,3.0,1.0,1.0
12186079,12186079,3965952,58526,100001,275225.0,2117-09-12,2117-09-13,1,hydralazine,HydrALAzine,...,HYDZ20I,283,1.747809e+10,1.0,4,1.0,4,3.0,1.0,1.0


Discussion:

- [x]  deprecate all entries except last one like labitem? (OBVIOUSLY MORE SIMPLE)
- [ ] add a new `DOSE_REPEAT` column whose numbers represent the repeat time of this drug during that timestep.

In [136]:
drop_indexs = []

for k, df_curr_hadmid in tqdm(gb_hadmid):
    for timestep in range(int(df_curr_hadmid.TIMESTEP.max()) + 1):
        df_curr_hadmid_curr_timestep = df_curr_hadmid[df_curr_hadmid.TIMESTEP == timestep]
        
        sr_ndc_value_counts = df_curr_hadmid_curr_timestep.NDC.value_counts()
        sr_ndc_repeat = sr_ndc_value_counts[sr_ndc_value_counts > 1]
        
        for ndc_repeat in list(sr_ndc_repeat.index):
            deprecate_entry_index = list(df_curr_hadmid_curr_timestep[df_curr_hadmid_curr_timestep.NDC == ndc_repeat]\
                                         .sort_values(by=["STARTDATE", "ENDDATE"])\
                                         .index[0:-1])
            drop_indexs.extend(deprecate_entry_index)
    if len(drop_indexs) > 10000:
        df_prescriptions_preprocessed.drop(drop_indexs, inplace=True)
        drop_indexs = []
        
df_prescriptions_preprocessed.drop(drop_indexs, inplace=True)

  0%|          | 0/50212 [00:00<?, ?it/s]

In [137]:
_gb_hadmid = df_prescriptions_preprocessed.groupby("HADM_ID")

def repeatability_check(df_curr_hadmid: pd.DataFrame):
    for timestep in range(int(df_curr_hadmid.TIMESTEP.max()) + 1):
        df_curr_hadmid_curr_timestep = df_curr_hadmid[df_curr_hadmid.TIMESTEP == timestep]
        sr_itemid_value_counts = df_curr_hadmid_curr_timestep.NDC.value_counts()
        sr_itemid_repeat = sr_itemid_value_counts[sr_itemid_value_counts > 1]
        assert len(sr_itemid_repeat) == 0
        
for k, df in tqdm(_gb_hadmid):
    repeatability_check(df)
    
df_prescriptions_preprocessed.to_csv(os.path.join(constant.PATH_MIMIC_III_ETL_OUTPUT, "PRESCRIPTIONS_PREPROCESSED.csv.gz"))

  0%|          | 0/50212 [00:00<?, ?it/s]

## Node feature

In [28]:
df_prescriptions_preprocessed = pd.read_csv(os.path.join(path_dataset, "PRESCRIPTIONS_PREPROCESSED.csv.gz"))
df_prescriptions_preprocessed.head()

/data/data2/041/envs/LERS2/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3269: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,...,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE,TIMESTEP
0,0,102562,492937,3951837,89132,198797,NaN,2151-06-27,2151-06-28,1,...,CYAN500,2341.0,8.770107e+10,1,1.0,11.0,1,1.0,3,0.0
1,1,245100,492946,3951840,89132,198797,NaN,2151-06-27,2151-06-28,1,...,CARA1,2766.0,6.373903e+10,1,1.0,4.0,1,1.0,3,0.0
2,2,1671716,11792392,3951836,89132,198797,NaN,2151-06-27,2151-06-28,1,...,ATEN25,15864.0,5.107908e+10,1,2.0,1.0,2,1.0,3,0.0
3,3,1835301,11792393,3951835,89132,198797,NaN,2151-06-27,2151-06-28,1,...,ALLO100,2535.0,5.107902e+10,1,1.0,1.0,1,1.0,3,0.0
4,4,2083587,492934,3951834,89132,198797,NaN,2151-06-27,2151-06-28,1,...,PRAV20,16367.0,5.107905e+10,1,1.0,1.0,1,1.0,2,0.0


In [29]:
gb_ndc_all = df_prescriptions_preprocessed.groupby("NDC")

### `DRUG_TYPE`

In [34]:
list_ndc_mix_feature_drug_type = []
for k, df in tqdm(gb_ndc_all):
    if len(df.DRUG_TYPE.value_counts()) > 1:
        list_ndc_mix_feature_drug_type.append(k)
        
list_ndc_mix_feature_drug_type

  0%|          | 0/4294 [00:00<?, ?it/s]

[18.0,
 22.0,
 23.0,
 25.0,
 29.0,
 70.0,
 87.0,
 96.0,
 2821501.0,
 6354120.0,
 6394132.0,
 9001104.0,
 9037301.0,
 9082501.0,
 9090020.0,
 13746686.0,
 74114102.0,
 74115112.0,
 74115170.0,
 74128301.0,
 74233211.0,
 74233326.0,
 74403101.0,
 74407532.0,
 74420101.0,
 74490223.0,
 74662502.0,
 74665106.0,
 74665305.0,
 74729501.0,
 74771503.0,
 74909431.0,
 74915801.0,
 74978901.0,
 78018003.0,
 78018201.0,
 186011291.0,
 186021003.0,
 186027613.0,
 186103301.0,
 186116502.0,
 338004903.0,
 338040360.0,
 338043103.0,
 409115170.0,
 409116202.0,
 409403101.0,
 409427001.0,
 409427701.0,
 409662502.0,
 409663734.0,
 409665106.0,
 409665305.0,
 409672903.0,
 409739172.0,
 469760040.0,
 517209625.0,
 517260225.0,
 517265025.0,
 517293025.0,
 517391025.0,
 641040025.0,
 641041425.0,
 641061025.0,
 641111834.0,
 703480503.0,
 781305714.0,
 942064104.0,
 17191003500.0,
 23731605305.0,
 49669521303.0,
 52769025105.0,
 55390003310.0,
 55390013110.0,
 55390023301.0,
 55513012610.0,
 5551301441

In [35]:
df_prescriptions_preprocessed[
    df_prescriptions_preprocessed.NDC == 2821501
].DRUG_TYPE.value_counts()

1    42845
3       25
Name: DRUG_TYPE, dtype: int64

In [36]:
df_prescriptions_preprocessed.DRUG_TYPE.value_counts()

1    8825724
2    1348851
3      15265
Name: DRUG_TYPE, dtype: int64

### `FORM_UNIT_DISP` 

Suitable as a node feature because it provides information about the packaging of the drug.

In [32]:
maxx_ndc_all_form_unit = 0
for k, df in tqdm(gb_ndc_all):
    maxx_ndc_all_form_unit = len(df.FORM_UNIT_DISP.value_counts()) if len(df.FORM_UNIT_DISP.value_counts()) > maxx_ndc_all_form_unit else maxx_ndc_all_form_unit
    
maxx_ndc_all_form_unit

  0%|          | 0/4294 [00:00<?, ?it/s]

5

### `DOSE_UNIT_RX`

It is not suitable as a node feature because it is related to the specific dosage of each medication; it is suitable as an edge feature.

In [33]:
maxx_ndc_all_dose_unit = 0
for k, df in tqdm(gb_ndc_all):
    maxx_ndc_all_dose_unit = len(df.DOSE_UNIT_RX.value_counts()) if len(df.DOSE_UNIT_RX.value_counts()) > maxx_ndc_all_dose_unit else maxx_ndc_all_dose_unit
    
maxx_ndc_all_dose_unit

  0%|          | 0/4294 [00:00<?, ?it/s]

10

In [41]:
df_drug_node_feature_per_ndc = pd.DataFrame(columns=["NDC", 
                                                     "DRUG_TYPE_MAIN_Proportion", 
                                                     "DRUG_TYPE_BASE_Proportion", 
                                                     "DRUG_TYPE_ADDITIVE_Proportion", 
                                                     "FORM_UNIT_DISP_Freq_1", 
                                                     "FORM_UNIT_DISP_Freq_2", 
                                                     "FORM_UNIT_DISP_Freq_3", 
                                                     "FORM_UNIT_DISP_Freq_4", 
                                                     "FORM_UNIT_DISP_Freq_5"])

for k, df in tqdm(gb_ndc_all):
    ser_drug_type = df.DRUG_TYPE.value_counts()
    sum_drug_type = ser_drug_type.sum()
    len_drug_type = len(ser_drug_type)
    
    ser_form_unit = df.FORM_UNIT_DISP.value_counts()
    len_form_unit = len(ser_form_unit)
    
    df_drug_node_feature_per_ndc.loc[len(df_drug_node_feature_per_ndc.index)] = [
        k,
        
        ser_drug_type.iloc[0] / sum_drug_type,
        ser_drug_type.iloc[1] / sum_drug_type if len_drug_type == 2 else 0,
        ser_drug_type.iloc[2] / sum_drug_type if len_drug_type == 3 else 0,

        ser_form_unit.index[0],
        ser_form_unit.index[1] if len_form_unit == 2 else 0,
        ser_form_unit.index[2] if len_form_unit == 3 else 0,
        ser_form_unit.index[3] if len_form_unit == 4 else 0,
        ser_form_unit.index[4] if len_form_unit == 5 else 0,
    ]
    
df_drug_node_feature_per_ndc.head()

  0%|          | 0/4294 [00:00<?, ?it/s]

,NDC,DRUG_TYPE_MAIN_Proportion,DRUG_TYPE_BASE_Proportion,DRUG_TYPE_ADDITIVE_Proportion,FORM_UNIT_DISP_Freq_1,FORM_UNIT_DISP_Freq_2,FORM_UNIT_DISP_Freq_3,FORM_UNIT_DISP_Freq_4,FORM_UNIT_DISP_Freq_5
0,1.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
1,2.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,4.0,1.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0
4,5.0,1.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0


In [39]:
with open(os.path.join(path_ddi_dataset, "ndc2rxnorm_mapping.txt"), 'r') as f:
    ndc2rxnorm = eval(f.read())
ndc2rxnorm

{'00338500341': '1665046',
 '00064100133': '545106',
 '00069077038': '616287',
 '00085043104': '746189',
 '58016069000': '201907',
 '00078012706': '104776',
 '00172541310': '197700',
 '51079037520': '905369',
 '51079089020': '198202',
 '00029315913': '261242',
 '00781144113': '198317',
 '00310032165': '1722941',
 '00002750201': '1719015',
 '00085177701': '213224',
 '00071400705': '855895',
 '00781143713': '865123',
 '00310027539': '153638',
 '00409909422': '1735003',
 '00087667217': '284499',
 '00378912398': '197696',
 '51079075920': '197380',
 '00436058532': '702519',
 '00703441211': '1863355',
 '51079089320': '197419',
 '51079076608': '1099563',
 '61703041463': '1721315',
 '00078033706': '261360',
 '00078033705': '261360',
 '00009000602': '859054',
 '61703041464': '1721314',
 '00172364910': '310490',
 '00009513502': '262091',
 '00009513503': '262091',
 '00004050109': '213177',
 '17236088901': '197366',
 '00087057007': '213633',
 '00087057009': '213633',
 '63824000840': '891301',
 '00

In [42]:
df_drug_node_feature_per_ndc.insert(len(df_drug_node_feature_per_ndc.columns), "rxnorm_id", np.NaN)

df_drug_node_feature_per_ndc_real = df_drug_node_feature_per_ndc[df_drug_node_feature_per_ndc.NDC > 1000]
df_drug_node_feature_per_ndc_fake = df_drug_node_feature_per_ndc[df_drug_node_feature_per_ndc.NDC < 1000]

df_drug_node_feature_per_ndc_real["NDC"] = df_drug_node_feature_per_ndc_real["NDC"].map("{:0>11.0f}".format)
df_drug_node_feature_per_ndc_real

/tmp/ipykernel_3849/847829642.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drug_node_feature_per_ndc_real["NDC"] = df_drug_node_feature_per_ndc_real["NDC"].map("{:0>11.0f}".format)


,NDC,DRUG_TYPE_MAIN_Proportion,DRUG_TYPE_BASE_Proportion,DRUG_TYPE_ADDITIVE_Proportion,FORM_UNIT_DISP_Freq_1,FORM_UNIT_DISP_Freq_2,FORM_UNIT_DISP_Freq_3,FORM_UNIT_DISP_Freq_4,FORM_UNIT_DISP_Freq_5,rxnorm_id
173,00002050101,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,NaN
174,00002140701,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,NaN
175,00002144401,1.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,NaN
176,00002202402,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN
177,00002322930,1.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
4289,78112001103,1.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,NaN
4290,79511050204,1.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,NaN
4291,87701071218,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN
4292,87701083336,1.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,NaN


In [53]:
df_drug_node_feature_per_ndc_final = pd.concat([df_drug_node_feature_per_ndc_real, 
                                                df_drug_node_feature_per_ndc_fake])
df_drug_node_feature_per_ndc_final["rxnorm_id"] = df_drug_node_feature_per_ndc_final["NDC"].map(ndc2rxnorm, na_action='ignore')
df_drug_node_feature_per_ndc_final

,NDC,DRUG_TYPE_MAIN_Proportion,DRUG_TYPE_BASE_Proportion,DRUG_TYPE_ADDITIVE_Proportion,FORM_UNIT_DISP_Freq_1,FORM_UNIT_DISP_Freq_2,FORM_UNIT_DISP_Freq_3,FORM_UNIT_DISP_Freq_4,FORM_UNIT_DISP_Freq_5,rxnorm_id
173,00002050101,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,207882
174,00002140701,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,853004
175,00002144401,1.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,208815
176,00002202402,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,313009
177,00002322930,1.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,352320
...,...,...,...,...,...,...,...,...,...,...
168,170.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,NaN
169,171.0,1.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,NaN
170,172.0,1.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,NaN
171,173.0,1.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,NaN


In [54]:
df_drug_node_feature_per_ndc_final[df_drug_node_feature_per_ndc_final.rxnorm_id.isnull()]

,NDC,DRUG_TYPE_MAIN_Proportion,DRUG_TYPE_BASE_Proportion,DRUG_TYPE_ADDITIVE_Proportion,FORM_UNIT_DISP_Freq_1,FORM_UNIT_DISP_Freq_2,FORM_UNIT_DISP_Freq_3,FORM_UNIT_DISP_Freq_4,FORM_UNIT_DISP_Freq_5,rxnorm_id
0,1.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,NaN
1,2.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,NaN
2,3.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,NaN
3,4.0,1.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,NaN
4,5.0,1.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
168,170.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,NaN
169,171.0,1.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,NaN
170,172.0,1.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,NaN
171,173.0,1.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,NaN


### These drugs indeed only have NDC and no RxNormID.

In [55]:
df_drug_node_feature_per_ndc_final[df_drug_node_feature_per_ndc_final.rxnorm_id.notnull()].iloc[-1]

NDC                              87701089415
DRUG_TYPE_MAIN_Proportion                1.0
DRUG_TYPE_BASE_Proportion                0.0
DRUG_TYPE_ADDITIVE_Proportion            0.0
FORM_UNIT_DISP_Freq_1                    8.0
FORM_UNIT_DISP_Freq_2                    0.0
FORM_UNIT_DISP_Freq_3                    0.0
FORM_UNIT_DISP_Freq_4                    0.0
FORM_UNIT_DISP_Freq_5                    0.0
rxnorm_id                                   
Name: 4293, dtype: object

In [56]:
df_drug_node_feature_per_ndc_final[df_drug_node_feature_per_ndc_final.rxnorm_id.notnull()].iloc[-1].rxnorm_id

''

In [57]:
df_drug_node_feature_per_ndc_final[df_drug_node_feature_per_ndc_final.rxnorm_id.notnull()].rxnorm_id.value_counts()

           20
206977     11
205964      6
311700      6
313323      6
           ..
199387      1
828576      1
905369      1
206800      1
1292751     1
Name: rxnorm_id, Length: 2998, dtype: int64

In [58]:
temp_df = df_drug_node_feature_per_ndc_final[df_drug_node_feature_per_ndc_final.rxnorm_id.notnull()]
temp_df[temp_df.rxnorm_id == '']

,NDC,DRUG_TYPE_MAIN_Proportion,DRUG_TYPE_BASE_Proportion,DRUG_TYPE_ADDITIVE_Proportion,FORM_UNIT_DISP_Freq_1,FORM_UNIT_DISP_Freq_2,FORM_UNIT_DISP_Freq_3,FORM_UNIT_DISP_Freq_4,FORM_UNIT_DISP_Freq_5,rxnorm_id
589,00026063203,1.000000,0.000000,0.0,5.0,0.0,0.0,0.0,0.0,
1524,00132000119,1.000000,0.000000,0.0,8.0,0.0,0.0,0.0,0.0,
1971,00338040360,0.844458,0.155542,0.0,3.0,2.0,0.0,0.0,0.0,
2742,08290002191,1.000000,0.000000,0.0,5.0,0.0,0.0,0.0,0.0,
2749,08881200441,1.000000,0.000000,0.0,5.0,0.0,0.0,0.0,0.0,
2812,10939003722,1.000000,0.000000,0.0,21.0,0.0,0.0,0.0,0.0,
2839,12539011501,1.000000,0.000000,0.0,7.0,0.0,0.0,0.0,0.0,
2850,15127020017,1.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,
2855,16563009802,1.000000,0.000000,0.0,8.0,0.0,0.0,0.0,0.0,
2857,16837085839,1.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,


In [66]:
# <https://lhncbc.nlm.nih.gov/RxNav/APIs/api-RxNorm.findRxcuiById.html>
url = 'https://rxnav.nlm.nih.gov/REST/rxcui.json?'
params = {
    "idtype": "NDC",
    "id": "55045122403"
}

r = requests.get(url, params=params)
print(r.status_code)
r.json()

200


{'idGroup': {}}

In [68]:
mask = df_drug_node_feature_per_ndc_final.rxnorm_id == ''
df_drug_node_feature_per_ndc_final.loc[mask, "rxnorm_id"] = np.NaN
df_drug_node_feature_per_ndc_final.loc[mask]

,NDC,DRUG_TYPE_MAIN_Proportion,DRUG_TYPE_BASE_Proportion,DRUG_TYPE_ADDITIVE_Proportion,FORM_UNIT_DISP_Freq_1,FORM_UNIT_DISP_Freq_2,FORM_UNIT_DISP_Freq_3,FORM_UNIT_DISP_Freq_4,FORM_UNIT_DISP_Freq_5,rxnorm_id
589,00026063203,1.000000,0.000000,0.0,5.0,0.0,0.0,0.0,0.0,NaN
1524,00132000119,1.000000,0.000000,0.0,8.0,0.0,0.0,0.0,0.0,NaN
1971,00338040360,0.844458,0.155542,0.0,3.0,2.0,0.0,0.0,0.0,NaN
2742,08290002191,1.000000,0.000000,0.0,5.0,0.0,0.0,0.0,0.0,NaN
2749,08881200441,1.000000,0.000000,0.0,5.0,0.0,0.0,0.0,0.0,NaN
2812,10939003722,1.000000,0.000000,0.0,21.0,0.0,0.0,0.0,0.0,NaN
2839,12539011501,1.000000,0.000000,0.0,7.0,0.0,0.0,0.0,0.0,NaN
2850,15127020017,1.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,NaN
2855,16563009802,1.000000,0.000000,0.0,8.0,0.0,0.0,0.0,0.0,NaN
2857,16837085839,1.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,NaN


In [69]:
mask = df_drug_node_feature_per_ndc_final.rxnorm_id == ''
df_drug_node_feature_per_ndc_final.loc[mask]

,NDC,DRUG_TYPE_MAIN_Proportion,DRUG_TYPE_BASE_Proportion,DRUG_TYPE_ADDITIVE_Proportion,FORM_UNIT_DISP_Freq_1,FORM_UNIT_DISP_Freq_2,FORM_UNIT_DISP_Freq_3,FORM_UNIT_DISP_Freq_4,FORM_UNIT_DISP_Freq_5,rxnorm_id


### save

In [77]:
df_drug_node_feature_per_ndc_final.to_csv(os.path.join(constant.PATH_MIMIC_III_ETL_OUTPUT, "DRUGS_NDC_FEAT.csv.gz"))